In [29]:
from pathlib import Path
from moviepy.editor import VideoFileClip
from pytubefix import YouTube
import os
import speech_recognition as sr
from pprint import pprint
import cv2
from PIL import Image
import torch
from semantic_router.encoders import VitEncoder
from semantic_chunkers import ConsecutiveChunker

# 0. Setting

In [2]:
# Base

# Video Downloading
# SET CONFIG
'''
video_url = "https://www.youtube.com/watch?v=7Hcg-rLYwdM"
output_video_path = "./video_data/"
output_folder = "./mixed_data/base/"
output_audio_path = "./mixed_data/output_audio.wav"

filepath = output_video_path + "input_vid.mp4"
Path(output_folder).mkdir(parents=True, exist_ok=True)
'''

'\nvideo_url = "https://www.youtube.com/watch?v=7Hcg-rLYwdM"\noutput_video_path = "./video_data/"\noutput_folder = "./mixed_data/base/"\noutput_audio_path = "./mixed_data/output_audio.wav"\n\nfilepath = output_video_path + "input_vid.mp4"\nPath(output_folder).mkdir(parents=True, exist_ok=True)\n'

In [3]:
# Semantic

# Video Downloading
# SET CONFIG
'''
video_url = "https://www.youtube.com/watch?v=7Hcg-rLYwdM"
output_video_path = "./video_data/"
output_folder = "./mixed_data/semantic/"
output_audio_path = "./mixed_data/semantic/output_audio.wav"

filepath = output_video_path + "input_vid.mp4"
Path(output_folder).mkdir(parents=True, exist_ok=True)
'''

'\nvideo_url = "https://www.youtube.com/watch?v=7Hcg-rLYwdM"\noutput_video_path = "./video_data/"\noutput_folder = "./mixed_data/semantic/"\noutput_audio_path = "./mixed_data/semantic/output_audio.wav"\n\nfilepath = output_video_path + "input_vid.mp4"\nPath(output_folder).mkdir(parents=True, exist_ok=True)\n'

In [4]:
#PySceneDetection

video_url = "https://www.youtube.com/watch?v=7Hcg-rLYwdM"
output_video_path = "./video_data/"
output_folder = "./mixed_data/psd/"
output_audio_path = "./mixed_data/psd/output_audio.wav"

filepath = output_video_path + "input_vid.mp4"
Path(output_folder).mkdir(parents=True, exist_ok=True)

# 1. Video Downloading

In [5]:
'''
def download_video(url, output_path):
    """
    Download a video from a given url and save it to the output path.

    Parameters:
    url (str): The url of the video to download.
    output_path (str): The path to save the video to.

    Returns:
    dict: A dictionary containing the metadata of the video.
    """

    yt = YouTube(url)
    metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}
    yt.streams.get_highest_resolution().download(
        output_path=output_path, filename="input_vid.mp4"
    )
    return metadata

metadata_vid = download_video(video_url, output_video_path)
'''

'\ndef download_video(url, output_path):\n    """\n    Download a video from a given url and save it to the output path.\n\n    Parameters:\n    url (str): The url of the video to download.\n    output_path (str): The path to save the video to.\n\n    Returns:\n    dict: A dictionary containing the metadata of the video.\n    """\n\n    yt = YouTube(url)\n    metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}\n    yt.streams.get_highest_resolution().download(\n        output_path=output_path, filename="input_vid.mp4"\n    )\n    return metadata\n\nmetadata_vid = download_video(video_url, output_video_path)\n'

# 2.Video Processing

## 2.1.1 Base Chunking

In [7]:
# 1 frame per 5 second
'''
def video_to_images(video_path, output_folder):
    """
    Convert a video to a sequence of images and save them to the output folder.

    Parameters:
    video_path (str): The path to the video file.
    output_folder (str): The path to the folder to save the images to.

    """
    clip = VideoFileClip(video_path)
    clip.write_images_sequence(
        os.path.join(output_folder, "base_frame%04d.png"), fps=0.2 #configure this for controlling frame rate.
    )
'''

'\ndef video_to_images(video_path, output_folder):\n    """\n    Convert a video to a sequence of images and save them to the output folder.\n\n    Parameters:\n    video_path (str): The path to the video file.\n    output_folder (str): The path to the folder to save the images to.\n\n    """\n    clip = VideoFileClip(video_path)\n    clip.write_images_sequence(\n        os.path.join(output_folder, "base_frame%04d.png"), fps=0.2 #configure this for controlling frame rate.\n    )\n'

## 2.1.2 Semantic Chunking

In [ ]:
# semanticly extract frame
'''
def video_to_images_semantic(video_path, output_folder, score_threshold = 0.4):
    vidcap = cv2.VideoCapture(video_path) #open the link

    # フレームを格納するリスト
    frames = [] 

    # フレームレートを取得
    fps = vidcap.get(cv2.CAP_PROP_FPS)  # フレームレートを取得
    frame_interval = int(fps / 5)  # 1秒に5枚のフレームを抽出するための間隔

    success, image = vidcap.read()  # 最初のフレームを読み込む
    frame_count = 0  # フレームのカウント

    while success:
        if frame_count % frame_interval == 0:  # 指定した間隔のフレームだけを抽出
            frames.append(image)
        success, image = vidcap.read()  # 次のフレームを読み込む
        frame_count += 1  # フレームカウントを増加

    # 抽出したフレームの数を表示
    print(f"Extracted {len(frames)} frames.")


    image_frames = list(map(Image.fromarray, frames))

    device = (
        "mps"
        if torch.backends.mps.is_available()
        else "cuda" if torch.cuda.is_available() else "cpu"
    )
    print(f"Using '{device}'")

    encoder = VitEncoder(device=device)
    chunker = ConsecutiveChunker(encoder=encoder, score_threshold = score_threshold)

    chunks = chunker(docs=[image_frames])
    #各chunkの中間にある写真を保存する

    for i, chunk in enumerate(chunks[0]):
        num_docs = len(chunk.splits)
        mid = num_docs // 2
        chunk.splits[mid].save(output_folder + str(i).zfill(4) + ".png")

# 2.1.3 PySceneDetect Chunking

In [8]:
def video_to_images_psd(video_path, output_folder):
    from scenedetect import open_video,SceneManager,StatsManager,save_images
    from scenedetect.detectors import ContentDetector

    output = output_folder
    video = open_video(video_path)

    scene_manager = SceneManager(stats_manager=StatsManager())
    scene_manager.add_detector(ContentDetector())
    scene_manager.detect_scenes(video)

    scene_list = scene_manager.get_scene_list()
    save_images(scene_list = scene_list, 
                video = video,
                image_extension = 'png',
                image_name_template = '$VIDEO_NAME-Scene-$SCENE_NUMBER',
                output_dir= output,
                num_images = 1)

## 2.2 Audio and Text

In [9]:
def video_to_audio(video_path, output_audio_path):
    """
    Convert a video to audio and save it to the output path.

    Parameters:
    video_path (str): The path to the video file.
    output_audio_path (str): The path to save the audio to.

    """
    clip = VideoFileClip(video_path)
    audio = clip.audio
    audio.write_audiofile(output_audio_path)

In [10]:
def audio_to_text(audio_path):
    """
    Convert an audio file to text.

    Parameters:
    audio_path (str): The path to the audio file.

    Returns:
    test (str): The text recognized from the audio.

    """
    recognizer = sr.Recognizer()
    audio = sr.AudioFile(audio_path)

    with audio as source:
        # Record the audio data
        audio_data = recognizer.record(source)

        try:
            # Recognize the speech
            text = recognizer.recognize_whisper(audio_data)
        except sr.UnknownValueError:
            print("Speech recognition could not understand the audio.")
        except sr.RequestError as e:
            print(f"Could not request results from service; {e}")

    return text

In [11]:
video_to_images_psd(filepath, output_folder)
video_to_audio(filepath, output_audio_path)
text_data = audio_to_text(output_audio_path)

with open(output_folder + "output_text.txt", "w") as file:
    file.write(text_data)
print("Text data saved to file")
file.close()
os.remove(output_audio_path)
print("Audio file removed")

MoviePy - Writing audio in ./mixed_data/psd/output_audio.wav


MoviePy - Done.
Text data saved to file
Audio file removed


In [31]:
text_data

" As I look back on the mission that we've had here on the International Space Station, I'm proud to have been a part of much of the science activities that happened over the last two months. I didn't think I would do another spacewalk and to now have the chance to have done four more was just icing on the cake for a wonderful mission. The 10th one, do you like the first one? No, a little more comfortable. It's hard to put into words just what it was like to be a part of this expedition, the Expedition 63. It'll be kind of a memory that will last a lifetime for me. It's been a true honor. Try and space X, Undock sequence commanded. The thrusters looking good. The hardest part was getting us launched, but the most important part is bringing us home. I've been trying that day. We love you. Hurry home for weeks and don't get my dog. Slash down. Welcome back to Planet Earth and thanks for flying SpaceX. We're literally on our own. Space dads are back on Earth after a 19-hour return journey

# 3.Building the Multi-Modal Index and Vector Store 

In [13]:
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.core import SimpleDirectoryReader, StorageContext

from llama_index.vector_stores.lancedb import LanceDBVectorStore
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

text_store = LanceDBVectorStore(uri="lancedb", table_name="text_collection")
image_store = LanceDBVectorStore(uri="lancedb", table_name="image_collection")
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)

# Create the MultiModal index
documents = SimpleDirectoryReader(output_folder).load_data()

index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [30]:
storage_context

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x178874070>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x17b354100>, vector_stores={'default': LanceDBVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, uri='lancedb', vector_column_name='vector', nprobes=20, refine_factor=None, text_key='text', doc_id_key='doc_id', api_key=None, region=None, mode='overwrite', query_type='vector', overfetch_factor=1), 'image': LanceDBVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, uri='lancedb', vector_column_name='vector', nprobes=20, refine_factor=None, text_key='text', doc_id_key='doc_id', api_key=None, region=None, mode='overwrite', query_type='vector', overfetch_factor=1)}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x17b3541c0>, property_graph_store=None)

# 4.Retrieving Relevant Images and Context

In [17]:
retriever_engine = index.as_retriever(
    similarity_top_k=5, image_similarity_top_k=5
)

In [14]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode


def retrieve(retriever_engine, query_str):
    retrieval_results = retriever_engine.retrieve(query_str)

    retrieved_image = []
    retrieved_text = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
            retrieved_text.append(res_node.text)

    return retrieved_image, retrieved_text

In [22]:
query_str = """
How many astronauts in this spaceship?
"""

In [23]:
img, txt = retrieve(retriever_engine=retriever_engine, query_str=query_str)
image_documents = SimpleDirectoryReader(
    input_dir=output_folder, input_files=img
).load_data()
context_str = "".join(txt)
#plot_images(img)

**Node ID:** 179d3abf-e219-41cf-870d-d41a0b19b119<br>**Similarity:** 0.6723143458366394<br>**Text:** As I look back on the mission that we've had here on the International Space Station, I'm proud to have been a part of much of the science activities that happened over the last two months. I didn'...<br>

In [24]:
img

['/Users/jyongzhi/Desktop/Video_RAG/multimodal RAG/mixed_data/psd/input_vid-Scene-002.png',
 '/Users/jyongzhi/Desktop/Video_RAG/multimodal RAG/mixed_data/psd/input_vid-Scene-012.png',
 '/Users/jyongzhi/Desktop/Video_RAG/multimodal RAG/mixed_data/psd/input_vid-Scene-001.png',
 '/Users/jyongzhi/Desktop/Video_RAG/multimodal RAG/mixed_data/psd/input_vid-Scene-008.png',
 '/Users/jyongzhi/Desktop/Video_RAG/multimodal RAG/mixed_data/psd/input_vid-Scene-009.png']

# 5.Reasoning and Response Generation

In [25]:
qa_tmpl_str = (
    """
 Given the provided information, including relevant images and retrieved context from the video, \
 accurately and precisely answer the query without any additional prior knowledge.\n"
    "Please ensure honesty and responsibility, refraining from any racist or sexist remarks.\n"
    "---------------------\n"
    "Context: {context_str}\n"
    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
"""
)

In [26]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

openai_mm_llm = OpenAIMultiModal(
    model="gpt-4o", api_key=os.getenv('OPENAI_API_KEY'), max_new_tokens=1500
)


response_1 = openai_mm_llm.complete(
    prompt=qa_tmpl_str.format(
        context_str=context_str, query_str=query_str, 
    ),
    image_documents=image_documents,
)

pprint(response_1.text)

'The spaceship had two astronauts on board.'


In [28]:
response_1.text

'The spaceship had two astronauts on board.'